### **Asssistants API Azure OpenAI**

In [1]:
import json
from openai import AzureOpenAI
import configparser
import time

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('default', 'AZURE_OPENAI_KEY')
api_endpoint = config.get('default','AZURE_OPENAI_ENDPOINT')

In [3]:
client = AzureOpenAI(
  azure_endpoint = api_endpoint, 
  api_key=api_key,  
  api_version="2024-02-15-preview"
)

In [21]:
assistant = client.beta.assistants.create(
    name="Algebra Tutor",
    instructions=f"You are a helpful AI assistant who helps with math questions and makes algebra visualizations."
                 f"You can create code in sandbox and run it, to answer those questions",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4"
)

In [22]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_6TVa2wiix4oYjvjVNLc81Nkz', created_at=1710373004, metadata={}, object='thread')


In [47]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Por favor crea una gráfica de la función y = x^2"
)

In [48]:
message

ThreadMessage(id='msg_aC3jShjwxbiStwOdkAf48F4K', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Por favor crea una gráfica de la función y = x^2'), type='text')], created_at=1710375028, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_6TVa2wiix4oYjvjVNLc81Nkz')

In [49]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [50]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [51]:
status = run.status

In [52]:
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    status = run.status

In [53]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [54]:
print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_76K86mg3QijmbqdKPoX0HW27",
      "assistant_id": "asst_SkJdYdfZgk2ClYQ8NHzs0h7Q",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-jiijKzc4nQLtMj4Pf5YWj647"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Aquí tienes el gráfico de la función $y = x^2$. Como puedes ver, es una parábola que se abre hacia arriba con el vértice en el origen (0,0)."
          },
          "type": "text"
        }
      ],
      "created_at": 1710375041,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_x5gBPp8A4ttUU23YSxe3BqHE",
      "thread_id": "thread_6TVa2wiix4oYjvjVNLc81Nkz"
    },
    {
      "id": "msg_aC3jShjwxbiStwOdkAf48F4K",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Por 

In [55]:
data = json.loads(messages.model_dump_json(indent=2)) 
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id) 

assistant-jiijKzc4nQLtMj4Pf5YWj647


In [56]:
content = client.files.content(image_file_id)

image= content.write_to_file("quadratic_function.png")

In [57]:
from PIL import Image

image = Image.open("quadratic_function.png")
image.show()

In [32]:
answer = json.loads(messages.model_dump_json(indent=2))

In [33]:
for item in answer['data']:
    print(f"{item['role']}: ")
    print(item['content'][0]['text']['value'])
    print ("\n")

assistant: 
Las soluciones para la ecuación $x^2 + 4 = 8$ son $x = -2$ y $x = 2$.


assistant: 
¡Por supuesto! Para resolver la ecuación cuadrática $x^2 + 4 = 8$, primero debemos restar $4$ de ambos lados para obtener $x^2 = 4$. Luego, podemos resolver para $x$ obteniendo la raíz cuadrada de ambos lados.

Lo haré usando Python.


user: 
¿Me puedes ayudar a resolver la ecuacón x^2 + 4 = 8?


